# Preparing the Dataset

### Content: 
- Mortality Data
- Population Data
- House Price Index Data
- Crosswalk Data
- Merging

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#load, wrangle, and clean each dataset
path = "/Users/kimballwightman/Documents/CareerFoundry/Covid-19 Housing Prices Analysis"

### Mortality Data

In [3]:
#county level COVID-19 mortality rate data
df_mortality = pd.read_csv(os.path.join(path,'02 Data','Original Data','us-counties.csv'))

In [4]:
df_mortality.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


We only want mortality rates (deaths) in 2020. Need to check to see if we need to filter out data from years after 2020.

In [5]:
#wrangling
df_mortality['date'].max()

'2022-03-31'

In [6]:
df_mortality['year'] = pd.DatetimeIndex(df_mortality['date']).year

In [7]:
df_mortality_2020 = df_mortality[df_mortality['year'] == 2020]

In [8]:
df_mortality_2020.head()

,date,county,state,fips,cases,deaths,year
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0,2020
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0,2020
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0,2020
3,2020-01-24,Cook,Illinois,17031.0,1,0.0,2020
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0,2020


In [9]:
df_mortality_2020.max()

date      2020-12-31
county       Ziebach
state        Wyoming
fips         78030.0
cases         770915
deaths       25144.0
year            2020
dtype: object

In [10]:
df_mortality_2020['county'].value_counts(dropna = False)

Washington              8777
Unknown                 7483
Jefferson               7343
Franklin                7007
Jackson                 6615
                        ... 
Petroleum                 88
Skagway Municipality      77
Esmeralda                 49
Loving                    45
Kalawao                   22
Name: county, Length: 1930, dtype: int64

In [11]:
#aggregate deaths by county
df_sum_deaths = df_mortality_2020.groupby(['fips','county','state','year']).agg({'deaths':['sum']})

In [12]:
df_sum_deaths

,,,,deaths
,,,,sum
fips,county,state,year,
1001.0,Autauga,Alabama,2020,5683.0
1003.0,Baldwin,Alabama,2020,12446.0
1005.0,Barbour,Alabama,2020,2035.0
1007.0,Bibb,Alabama,2020,2687.0
1009.0,Blount,Alabama,2020,3855.0
...,...,...,...,...
72151.0,Yabucoa,Puerto Rico,2020,0.0
72153.0,Yauco,Puerto Rico,2020,0.0


In [13]:
df_sum_deaths = df_sum_deaths.reset_index()

In [14]:
df_sum_deaths.head()

,fips,county,state,year,deaths
,,,,,sum
0,1001.0,Autauga,Alabama,2020,5683.0
1,1003.0,Baldwin,Alabama,2020,12446.0
2,1005.0,Barbour,Alabama,2020,2035.0
3,1007.0,Bibb,Alabama,2020,2687.0
4,1009.0,Blount,Alabama,2020,3855.0


In [15]:
df_sum_deaths = df_sum_deaths.droplevel(1, axis = 1)

In [16]:
df_sum_deaths.head()

,fips,county,state,year,deaths
0,1001.0,Autauga,Alabama,2020,5683.0
1,1003.0,Baldwin,Alabama,2020,12446.0
2,1005.0,Barbour,Alabama,2020,2035.0
3,1007.0,Bibb,Alabama,2020,2687.0
4,1009.0,Blount,Alabama,2020,3855.0


In [17]:
df_sum_deaths.dtypes

fips      float64
county     object
state      object
year        int64
deaths    float64
dtype: object

### Population Data

In [18]:
df_population = pd.read_excel(os.path.join(path,'02 Data','Original Data','2020pop.xlsx'))

In [19]:
df_population.head()

,ID,badid,County Full,PopTotal
0,1001,0500000US01001,"Autauga County, Alabama",58805
1,1003,0500000US01003,"Baldwin County, Alabama",231767
2,1005,0500000US01005,"Barbour County, Alabama",25223
3,1007,0500000US01007,"Bibb County, Alabama",22293
4,1009,0500000US01009,"Blount County, Alabama",59134


In [20]:
#wrangling
df_population.dtypes

ID              int64
badid          object
County Full    object
PopTotal        int64
dtype: object

In [21]:
df_population = df_population.rename(columns = {'ID':'fips'})

In [22]:
df_population.head()

,fips,badid,County Full,PopTotal
0,1001,0500000US01001,"Autauga County, Alabama",58805
1,1003,0500000US01003,"Baldwin County, Alabama",231767
2,1005,0500000US01005,"Barbour County, Alabama",25223
3,1007,0500000US01007,"Bibb County, Alabama",22293
4,1009,0500000US01009,"Blount County, Alabama",59134


### House Price Index Data

In [23]:
df_houseprice = pd.read_excel(os.path.join(path,'02 Data','Original Data','HPI_AT_3zip.xlsx'))

In [24]:
df_houseprice.head()

,Three-Digit ZIP Code,Year,Quarter,Index (NSA),Index Type
0,10,1995,1,100.00,Native 3-Digit ZIP index
1,10,1995,2,101.46,Native 3-Digit ZIP index
2,10,1995,3,103.70,Native 3-Digit ZIP index
3,10,1995,4,103.65,Native 3-Digit ZIP index
4,10,1996,1,105.03,Native 3-Digit ZIP index


In [25]:
#wrangling
df_hp = df_houseprice[df_houseprice['Year'].isin([2020,2021])]

In [26]:
df_hp['Year'].min()

2020

In [27]:
df_hp['Year'].max()

2021

In [28]:
#mean of house price index by zip code and year
df_hp_mean = df_hp.groupby(['Three-Digit ZIP Code','Year']).agg({'Index (NSA)':['mean']})

In [29]:
df_hp_mean.head()

Index (NSA)
                                 mean
Three-Digit ZIP Code Year            
10                   2020    224.2225
                     2021    254.5000
11                   2020    237.3775
                     2021    276.9225
12                   2020    234.7175

In [30]:
df_hp_mean = df_hp_mean.reset_index()

In [31]:
df_hp_mean = df_hp_mean.droplevel(1, axis = 1)

In [32]:
df_hp_mean.head()

,Three-Digit ZIP Code,Year,Index (NSA)
0,10,2020,224.2225
1,10,2021,254.5000
2,11,2020,237.3775
3,11,2021,276.9225
4,12,2020,234.7175


In [33]:
#reshape long to wide for index score by year
df_hp_wide = df_hp_mean.pivot('Three-Digit ZIP Code','Year')

In [34]:
df_hp_wide.head()

Index (NSA)          
Year                        2020      2021
Three-Digit ZIP Code                      
10                      224.2225  254.5000
11                      237.3775  276.9225
12                      234.7175  271.1075
13                      226.2875  257.8325
14                      260.9150  300.5450

In [35]:
df_hp_wide = df_hp_wide.reset_index()

In [36]:
df_hp_wide.head()

Three-Digit ZIP Code Index (NSA)          
Year                             2020      2021
0                      10    224.2225  254.5000
1                      11    237.3775  276.9225
2                      12    234.7175  271.1075
3                      13    226.2875  257.8325
4                      14    260.9150  300.5450

In [37]:
df_hp_wide.columns

MultiIndex([('Three-Digit ZIP Code',   ''),
            (         'Index (NSA)', 2020),
            (         'Index (NSA)', 2021)],
           names=[None, 'Year'])

In [38]:
#generate the change in index score from 2020 to 2021
df_hp_wide['index change'] = df_hp_wide[('Index (NSA)', 2021)] - df_hp_wide[('Index (NSA)', 2020)]

In [39]:
df_hp_wide.head()

Three-Digit ZIP Code Index (NSA)           index change
Year                             2020      2021             
0                      10    224.2225  254.5000      30.2775
1                      11    237.3775  276.9225      39.5450
2                      12    234.7175  271.1075      36.3900
3                      13    226.2875  257.8325      31.5450
4                      14    260.9150  300.5450      39.6300

In [40]:
df_hp_wide.columns

MultiIndex([('Three-Digit ZIP Code',   ''),
            (         'Index (NSA)', 2020),
            (         'Index (NSA)', 2021),
            (        'index change',   '')],
           names=[None, 'Year'])

In [41]:
#reshape back to long format
df_hp_long = pd.melt(df_hp_wide, id_vars = ['Three-Digit ZIP Code','index change'], value_name = 'Index (NSA)')


/Users/kimballwightman/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [42]:
df_hp_long.head()

,Three-Digit ZIP Code,index change,NaN,Year,Index (NSA)
0,10,30.2775,Index (NSA),2020,224.2225
1,11,39.5450,Index (NSA),2020,237.3775
2,12,36.3900,Index (NSA),2020,234.7175
3,13,31.5450,Index (NSA),2020,226.2875
4,14,39.6300,Index (NSA),2020,260.9150


In [43]:
df_hp_2021 = df_hp_long[df_hp_long['Year'] == 2021]

In [44]:
df_hp_2021 = df_hp_2021.loc[:, df_hp_2021.columns.notna()]

In [45]:
df_hp_2021 = df_hp_2021.drop(columns = ['Year','Index (NSA)'])

In [46]:
df_hp_2021.rename(columns = {'Three-Digit ZIP Code':'zip_three'}, inplace = True)

In [47]:
df_hp_2021.head()

,zip_three,index change
882,10,30.2775
883,11,39.5450
884,12,36.3900
885,13,31.5450
886,14,39.6300


In [48]:
df_hp_2021.shape

(882, 2)

In [49]:
df_hp_2021.dtypes

zip_three         int64
index change    float64
dtype: object

In [50]:
df_hp_2021['zip_three'].astype('str')

882      10
883      11
884      12
885      13
886      14
       ... 
1759    995
1760    996
1761    997
1762    998
1763    999
Name: zip_three, Length: 882, dtype: object

### Crosswalk Data

In [51]:
df_countyzip = pd.read_excel(os.path.join(path,'02 Data','Original Data','COUNTY_ZIP_032020.xlsx'))

In [52]:
df_countyzip.head()

,COUNTY,ZIP
0,1001,36749
1,1001,36068
2,1001,36703
3,1001,36066
4,1001,36051


In [53]:
df_countyzip.dtypes

COUNTY    int64
ZIP       int64
dtype: object

In [54]:
df_countyzip['ZIP'] = df_countyzip['ZIP'].astype('str')

In [55]:
df_countyzip['zip_three'] = df_countyzip['ZIP'].str.slice(0,3)

In [56]:
df_countyzip['zip_three'] = df_countyzip['zip_three'].astype('int64')

In [57]:
df_countyzip.head()

,COUNTY,ZIP,zip_three
0,1001,36749,367
1,1001,36068,360
2,1001,36703,367
3,1001,36066,360
4,1001,36051,360


In [58]:
df_countyzip = df_countyzip.rename(columns = {'COUNTY':'fips'})

In [59]:
df_countyzip = df_countyzip.drop(columns = ['ZIP'])

In [60]:
df_countyzip.head()

,fips,zip_three
0,1001,367
1,1001,360
2,1001,367
3,1001,360
4,1001,360


In [61]:
df_cz_dups = df_countyzip[df_countyzip.duplicated()]

In [62]:
df_cz_dups

,fips,zip_three
2,1001,367
3,1001,360
4,1001,360
5,1001,360
6,1001,360
...,...,...
53840,56045,827
53841,56045,827
53843,60010,969
53844,60010,969


In [63]:
df_cz_nodups = df_countyzip.drop_duplicates()

In [64]:
df_cz_nodups.shape

(8046, 2)

### Merging

In [65]:
#merge mortality data and population data
df_merge1 = pd.merge(df_sum_deaths, df_population, on = ['fips'])

In [66]:
df_merge1

,fips,county,state,year,deaths,badid,County Full,PopTotal
0,1001.0,Autauga,Alabama,2020,5683.0,0500000US01001,"Autauga County, Alabama",58805
1,1003.0,Baldwin,Alabama,2020,12446.0,0500000US01003,"Baldwin County, Alabama",231767
2,1005.0,Barbour,Alabama,2020,2035.0,0500000US01005,"Barbour County, Alabama",25223
3,1007.0,Bibb,Alabama,2020,2687.0,0500000US01007,"Bibb County, Alabama",22293
4,1009.0,Blount,Alabama,2020,3855.0,0500000US01009,"Blount County, Alabama",59134
...,...,...,...,...,...,...,...,...
3127,56037.0,Sweetwater,Wyoming,2020,768.0,0500000US56037,"Sweetwater County, Wyoming",42272
3128,56039.0,Teton,Wyoming,2020,317.0,0500000US56039,"Teton County, Wyoming",23331
3129,56041.0,Uinta,Wyoming,2020,502.0,0500000US56041,"Uinta County, Wyoming",20450
3130,56043.0,Washakie,Wyoming,2020,1397.0,0500000US56043,"Washakie County, Wyoming",7685


In [67]:
#merge house price index data and crosswalk data
df_merge2 = pd.merge(df_hp_2021, df_cz_nodups, on = ['zip_three'])

In [68]:
df_merge2

,zip_three,index change,fips
0,100,26.0150,25003
1,100,26.0150,25011
2,100,26.0150,25013
3,100,26.0150,25015
4,100,26.0150,25027
...,...,...,...
7862,998,20.3975,2220
7863,998,20.3975,2230
7864,999,20.3975,2130
7865,999,20.3975,2198


In [69]:
#combine all into one dataset
df_merge_final = pd.merge(df_merge1, df_merge2, on = ['fips'])

In [70]:
df_merge_final

,fips,county,state,year,deaths,badid,County Full,PopTotal,zip_three,index change
0,1001.0,Autauga,Alabama,2020,5683.0,0500000US01001,"Autauga County, Alabama",58805,360,16.7625
1,1001.0,Autauga,Alabama,2020,5683.0,0500000US01001,"Autauga County, Alabama",58805,367,23.9850
2,1003.0,Baldwin,Alabama,2020,12446.0,0500000US01003,"Baldwin County, Alabama",231767,364,23.9850
3,1003.0,Baldwin,Alabama,2020,12446.0,0500000US01003,"Baldwin County, Alabama",231767,365,34.1150
4,1005.0,Barbour,Alabama,2020,2035.0,0500000US01005,"Barbour County, Alabama",25223,360,16.7625
...,...,...,...,...,...,...,...,...,...,...
7541,56039.0,Teton,Wyoming,2020,317.0,0500000US56039,"Teton County, Wyoming",23331,830,32.7400
7542,56039.0,Teton,Wyoming,2020,317.0,0500000US56039,"Teton County, Wyoming",23331,834,70.1175
7543,56041.0,Uinta,Wyoming,2020,502.0,0500000US56041,"Uinta County, Wyoming",20450,829,22.4700
7544,56043.0,Washakie,Wyoming,2020,1397.0,0500000US56043,"Washakie County, Wyoming",7685,824,32.7400


In [71]:
#create a state fips code to create dummy variables by state
df_merge_final['fips'] = df_merge_final['fips'].astype('str')

In [72]:
df_merge_final.dtypes

fips             object
county           object
state            object
year              int64
deaths          float64
badid            object
County Full      object
PopTotal          int64
zip_three         int64
index change    float64
dtype: object

In [73]:
df_merge_final['fips_length'] = df_merge_final['fips'].apply(len)

In [74]:
df_merge_final['fips_length'].value_counts()

7    6749
6     797
Name: fips_length, dtype: int64

As we can see here, the fips codes that start with a 0 are missing that 0 in the given value. We need to attach the 0 back on to these fips codes so we can extract the first two digits, which signify the state code for that county.

In [75]:
def fips_length (row):
    if row['fips_length'] == 6:
        return '0'+row['fips']
    else: return row['fips']

df_merge_final['state_code'] = df_merge_final.apply(fips_length, axis = 1)

In [76]:
df_merge_final['state_fips'] = df_merge_final['state_code'].str.slice(0,2)

In [77]:
df_merge_final = df_merge_final.drop(columns = ['state_code','fips_length','year','badid','County Full'])

In [78]:
df_merge_final.head()

,fips,county,state,deaths,PopTotal,zip_three,index change,state_fips
0,1001.0,Autauga,Alabama,5683.0,58805,360,16.7625,01
1,1001.0,Autauga,Alabama,5683.0,58805,367,23.9850,01
2,1003.0,Baldwin,Alabama,12446.0,231767,364,23.9850,01
3,1003.0,Baldwin,Alabama,12446.0,231767,365,34.1150,01
4,1005.0,Barbour,Alabama,2035.0,25223,360,16.7625,01


In [79]:
df_merge_final['state_fips'].value_counts().sort_index()

01    161
02     32
04     37
05    153
06    174
08    115
09    125
10      5
11      1
12    132
13    270
15      6
16     59
17    236
18    214
19    264
20    226
21    264
22    108
23    185
24     49
25    230
26    173
27    171
28    169
29    262
30     80
31    183
32     27
33    106
34    274
35     62
36    198
37    207
38     96
39    246
40    166
41     62
42    231
44     34
45    101
46     98
47    202
48    482
49     39
50    125
51    285
53     80
54    156
55    149
56     36
Name: state_fips, dtype: int64

In [80]:
#generate a deaths per capita variable
df_merge_final['deaths_pc'] = (df_merge_final['deaths']/df_merge_final['PopTotal'])*100000

In [81]:
df_merge_final.head(20)

,fips,county,state,deaths,PopTotal,zip_three,index change,state_fips,deaths_pc
0,1001.0,Autauga,Alabama,5683.0,58805,360,16.7625,01,9664.144205
1,1001.0,Autauga,Alabama,5683.0,58805,367,23.9850,01,9664.144205
2,1003.0,Baldwin,Alabama,12446.0,231767,364,23.9850,01,5370.048368
3,1003.0,Baldwin,Alabama,12446.0,231767,365,34.1150,01,5370.048368
4,1005.0,Barbour,Alabama,2035.0,25223,360,16.7625,01,8068.033144
5,1005.0,Barbour,Alabama,2035.0,25223,363,17.2300,01,8068.033144
6,1007.0,Bibb,Alabama,2687.0,22293,350,24.3175,01,12053.110842
7,1007.0,Bibb,Alabama,2687.0,22293,351,24.7100,01,12053.110842
8,1007.0,Bibb,Alabama,2687.0,22293,354,19.2275,01,12053.110842
9,1007.0,Bibb,Alabama,2687.0,22293,367,23.9850,01,12053.110842


In [83]:
df_merge_final.to_pickle(os.path.join(path,'02 Data','Prepared Data','df_merge.pkl'))